# Linear Regression

### Pytorch Realisation of Linear Regression Algorithm

In [1]:
import torch
from torch.nn import Linear, functional as F, Module
from torch.optim import SGD

In [2]:
class LinearRegression(Module):
    def __init__(self, in_features, out_features = 1, bias = True):
        super().__init__()
        self.linear = Linear(in_features= in_features, out_features=out_features, bias=bias)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-5)
    
    def fit(self, train_dl, num_epochs=100):
        for epoch in range(num_epochs+1):
            for x_batch, y_batch in train_dl:
                prediction = self.linear(x_batch.float())
                loss = F.mse_loss(prediction, y_batch.reshape(prediction.shape).float())
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {loss}')
        

<br>

#### Test on Boston Houses Dataset

In [3]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('data.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506,0.98765,0.0,12.50,0,0.561,6.980,89.0,2.0980,3,320,23.0,396.00,12.00,12.0
507,0.23456,0.0,12.50,0,0.561,6.980,76.0,2.6540,3,320,23.0,343.00,25.00,32.0
508,0.44433,0.0,12.50,0,0.561,6.123,98.0,2.9870,3,320,23.0,343.00,21.00,54.0
509,0.77763,0.0,12.70,0,0.561,6.222,34.0,2.5430,3,329,23.0,343.00,76.00,67.0


In [5]:
if df.isna().sum().sum() > 0:
    df.dropna(inplace=True)

In [6]:
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

In [7]:
X = df[df.columns[:-1]].values
y = df[df.columns[-1]].values

In [8]:
X_ = torch.from_numpy(X)
y_ = torch.from_numpy(y)

In [9]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

tensor_dataset = TensorDataset(X_, y_)
data_loader = DataLoader(tensor_dataset, batch_size=5, shuffle = True)

In [10]:
l = LinearRegression(13, 1)

In [11]:
l.fit(data_loader, num_epochs=100)

c:\users\user\appdata\local\programs\python\python37\lib\site-packages\torch\autograd\__init__.py:156: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 6050). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:112.)
  allow_unreachable=True, accumulate_grad=True)  # allow_unreachable flag


Epoch 0, Loss: 2908.119384765625
Epoch 10, Loss: 843.19580078125
Epoch 20, Loss: 21776.533203125
Epoch 30, Loss: 3332.93701171875
Epoch 40, Loss: 841.0799560546875
Epoch 50, Loss: 1.6738157272338867
Epoch 60, Loss: 13.713883399963379
Epoch 70, Loss: 43.89979934692383
Epoch 80, Loss: 10066.51171875
Epoch 90, Loss: 148.62298583984375
Epoch 100, Loss: 144.6781768798828


<br><br>

### Numpy Realisation of Linear Regression Algorithm

In [12]:
class LinearRegression_numpy():
    def __init__(self, in_features, out_features, bias = True):
        self.weight = np.random.rand(in_features, out_features)
        self.bias = np.random.rand(out_features)
    
    def __loss_fn(self, y_predicted, target):
        return np.square(y_predicted - target)
    
    def predict(self, X):
        return np.matmul(X, self.weight) + self.bias
    
    def fit(self, X, y, num_epochs, lr):
        mean_loss = 0
        for epoch in range(num_epochs):
            for index in range(len(X)):
                x_value, y_value = X[index], y[index]
                loss = self.__loss_fn(self.predict(x_value), y_value) 
                mean_loss += loss
                
                direction = 2 * (self.predict(x_value) - y_value) 
                self.weight -= lr * (direction * x_value.transpose()).reshape(self.weight.shape)
                self.bias -= lr * direction
            
            if epoch % 10 == 0:
                print(f'Epoch {epoch}, Loss: {mean_loss/(10 * len(X))}')
                mean_loss = 0
    
    def score(self, X, y):
        mean_loss = 0
        for index in range(len(X)):
            x_value, y_value = X[index], y[index]
            loss = self.__loss_fn(self.predict(x_value), y_value) 
            mean_loss += np.sqrt(loss)
        return mean_loss / len(X)

In [13]:
w = LinearRegression_numpy(13, 1)

In [14]:
w.fit(X, y, 100, 5e-7)

Epoch 0, Loss: [106.63241068]
Epoch 10, Loss: [80.64286182]
Epoch 20, Loss: [57.62909224]
Epoch 30, Loss: [51.99977692]
Epoch 40, Loss: [48.65352246]
Epoch 50, Loss: [46.27127407]
Epoch 60, Loss: [44.52092759]
Epoch 70, Loss: [43.22622745]
Epoch 80, Loss: [42.26375704]
Epoch 90, Loss: [41.5429833]


In [15]:
w.score(X,y)

array([17.11504683])

In [16]:
w.weight

array([[ 0.02196479],
       [ 0.05235669],
       [ 0.45523643],
       [ 0.70789005],
       [ 0.55895797],
       [ 1.06674107],
       [-0.03030725],
       [ 0.10108925],
       [ 0.41573908],
       [-0.00368688],
       [ 0.74765274],
       [ 0.03675027],
       [-0.21971944]])